<a href="https://colab.research.google.com/github/jslee0110/.2023-1aidl/blob/main/light_gbm%2Bxgboost_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import warnings
import gc

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

from imblearn.under_sampling import RandomUnderSampler

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
import lightgbm as lgb

warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
test=pd.read_csv('drive/MyDrive/data/test.csv')

In [4]:
train = pd.read_csv('drive/MyDrive/deleted_train1.csv')

In [5]:
train.head()

,id,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
0,1,1,1,0.538462,0,2,1,0.968248,0.154321,0.961938,1,0.333333
1,2,0,1,0.269231,1,1,0,0.600141,0.932099,0.844291,0,0.083333
2,4,0,1,0.288462,1,0,0,0.492679,0.932099,0.982699,0,0.250000
3,5,0,1,0.903846,1,1,0,0.425631,0.932099,0.647059,0,0.166667
4,6,1,1,0.865385,1,1,0,0.407603,0.932099,0.622837,0,0.000000


In [6]:
train.drop(['id'],axis=1)

,Gender,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AgeGroup
0,1,1,0.538462,0,2,1,0.968248,0.154321,0.961938,1,0.333333
1,0,1,0.269231,1,1,0,0.600141,0.932099,0.844291,0,0.083333
2,0,1,0.288462,1,0,0,0.492679,0.932099,0.982699,0,0.250000
3,0,1,0.903846,1,1,0,0.425631,0.932099,0.647059,0,0.166667
4,1,1,0.865385,1,1,0,0.407603,0.932099,0.622837,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
9127520,1,1,0.538462,1,0,0,0.457806,0.944444,0.660900,0,0.166667
9127521,1,1,0.115385,0,0,1,0.412612,0.154321,0.719723,0,0.416667
9127522,0,1,0.692308,0,1,1,0.449603,0.932099,0.363322,1,0.083333
9127523,0,1,0.538462,0,0,1,0.783595,0.154321,0.913495,1,0.500000


In [7]:
cat_cols = ["Gender", "Driving_License", "Previously_Insured", "Vehicle_Age", "Vehicle_Damage"]
num_cols = ["AgeGroup", "Region_Code", "Annual_Premium", "Policy_Sales_Channel", "Vintage"]
target = "Response"

In [8]:
X = train[cat_cols + num_cols]



y = train[target]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

In [10]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np

# XGBoost 모델 설정 및 훈련
xgb_clf = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='auc',
    n_estimators=1500,
    learning_rate=0.02,
    max_depth=6,
    subsample=0.85,
    colsample_bytree=0.75,
    gamma=0.2,
    min_child_weight=1,
    reg_alpha=0.01,
    reg_lambda=0.8,
    tree_method='gpu_hist'  # device="gpu"는 더 이상 사용되지 않음, 대신 tree_method 사용
)

xgb_clf.fit(X_train, y_train)
xgb_pred = xgb_clf.predict_proba(X_test)[:, 1]

xgb_auc = roc_auc_score(y_test, xgb_pred)
print(f"XGBoost AUC: {xgb_auc}")



XGBoost AUC: 0.8834478895157484


TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

In [11]:
# LightGBM 모델 설정 및 훈련
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',  # Change metric to logloss for probability predictions
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Early stopping을 위한 callback 설정
callbacks = [lgb.early_stopping(stopping_rounds=10)]

bst = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data], callbacks=callbacks)

y_prob = bst.predict(X_test, num_iteration=bst.best_iteration)
lgb_auc = roc_auc_score(y_test, y_prob)
print(f'LightGBM AUC: {lgb_auc}')

# XGBoost와 LightGBM 예측 결과를 앙상블
ensemble_pred = (xgb_pred + y_prob) / 2

# 앙상블 모델의 AUC 계산
ensemble_auc = roc_auc_score(y_test, ensemble_pred)
fpr, tpr, thresholds = roc_curve(y_test, ensemble_pred)

print(f'Ensemble ROC AUC: {ensemble_auc}')


[LightGBM] [Info] Number of positive: 882941, number of negative: 6419079
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.111570 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 7302020, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120917 -> initscore=-1.983772
[LightGBM] [Info] Start training from score -1.983772
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.249263
LightGBM AUC: 0.8791333944228049
Ensemble ROC AUC: 0.8822548444031066
